# Handling Missing Data

Why can a dataset have missing values? Sometimes an observation could not be recorded for whatever reason, like a sensor or instrument being broken or a survey respondent chose to not answer a question. Unfortunately, machine learning and statistical models often do not cope well with missing values. For this reason you will likely consider removing them. 

In this section, we will cover how to identify and remove data related to missing values.

> Please be advised you should fully understand why values are missing and trace back to the source of what produced the data. You should also be mindful of any selection biases that might emerge because of the missing data. For example, if you remove survey respondent records that did not answer a certain question... that can bias your models towards the population that chose to answer! Sometimes it is more interesting to ask why values are missing than just ignoring them. 

To get setup, let's bring in a SQLite database and extract the data from the `WEATHER_MONITOR` table. However, we will only analyze on region of stations by getting only records with `LOCATION_ID` of `2`,`28`, or `48`, and for the month of April. We will have SQL do that filtering. 

In [ ]:
import urllib.request
import sqlite3
import pandas as pd 

# Download SQLite database and connect to it 
urllib.request.urlretrieve("https://github.com/thomasnield/machine-learning-demo-data/blob/master/relational/company_operations.db?raw=true", "company_operations.db") 

conn = sqlite3.connect('company_operations.db')

df = pd.read_sql("""
SELECT * FROM WEATHER_MONITOR 
WHERE LOCATION_ID IN (2,28,48) 
AND strftime('%m', REPORT_DATE) = '04'

""", conn, parse_dates=['REPORT_DATE'])
df

> If you want to learn more about SQL, check out the [Anaconda course here](https://learning.anaconda.cloud/introduction-to-sql).

Notice above that the `RAIN` column has `NaN` values, meaing those values are `None` and missing. There is also one record where a `TEMPERATURE` value is mssing. We are going to learn a few techniques on how to identify and handle these. 

## Tracking Missing Values

To find missing values, we can use the `isna()` function on a dataframe. 

In [ ]:
df.isna()

We can also use the `any()` function to see which columns contain any missing values.

In [ ]:
df.isna().any()

You can also flip the axis for `any()` and get a boolean series of whether each row contains a missing value. 

In [ ]:
df.isna().any(axis=1)

Note there is also a `notna()` counterpart which will flip the condition and set provided values to `True` and missing values to `False`. There are also aliases `isnull()` and `notnull()` which are just different names for the same operations.

Of course, we can take that boolean series and pass it to the `loc` getter to retrieve those columns with `NaN` values. 

In [ ]:
df.loc[:, df.isna().any()]

To find missing values for specific columns, we can use a filtering operation using logical operators. Here we find all records where there was a missing `TEMPERATURE` or `RAIN` value.  

In [ ]:
df[df['TEMPERATURE'].isna() | df['RAIN'].isna()]

We can also filter for all records containing any missing values across all fields.

In [ ]:
df[df.isna().any(axis=1)]

## Removing Rows with Missing Values

As stated earlier, many machine learning and statistical models do not tolerate `NA`, `NaN`, or other missing null values. If you understand why they are missing, and do not think their absence will bias your model significantly, then you can simply remove those records with missing values. 

You can use the `drop()` operator with conditional logic as we learned in previous sections, but there is also a handy `dropna()` function just for this purpose. 

Below we use `dropna()` to remove all records with `NaN` values. Note I am not using the `inplace=True` parameter here so I can demonstrate other examples later. 

In [ ]:
df.dropna(axis=0) # use inplace=True to replace current dataframe 

Notice those four records with `NaN` values for `RAIN` or `TEMPERATURE` are now gone. We can also provide only a `subset` of indices to consider for dropping null values. Below, we only drop records where `NA` exists in the `RAIN` column.  

In [ ]:
df.dropna(axis=0, subset=["RAIN"])

If we would rather drop those columns with `NaN` values instead, we can use `axis=1`. 

In [ ]:
df.dropna(axis=1)

## Replacing Missing Values

While this may not make sense from a machine learning perspective, there may be times you want to replace missing values. You can do this by using teh `fillna()` function. Below we replace all `na` values in our table with `-1`. Unfortunately, there is no `subset` paramter for this function so to target specific columns you will need to extract them out, apply the `fillna()` function, and then assign them back. 

In [ ]:
df.fillna(value=-1, inplace=True)
df

> There are other methods that `fillna()` can do to fill in missing values. [Be sure to read the Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna) to learn more. 

Conversely there may be times you want to replace certain values with `na`, such as empty strings or placeholder strings like 'NULL'. We just turned the `NaN` values into `-1`. Let's convert them back to `NaN` using the `replace()` function.

In [ ]:
from numpy import nan

df.replace(-1, nan, inplace=True)
df

## Fill in Missing Values with Mean

Another way to cope with missing values that might be more agreeable for machine learning and statistical models is to use a statistical value replacement such as a `mean` or `median`. 

Let's bring in the `SimpleImputer` from scikit-learn and set it to use the `mean`. 

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

Let's then apply the `TEMPERATURE` and `RAIN` fields to the imputer. 

In [ ]:
transform_input = df[['TEMPERATURE','RAIN']]

imputer.fit(transform_input)

The `SimpleImputer` will calculate the mean for `RAIN` to be `0.8585` and for `TEMPERATURE` to be `59.740909`. We can then apply these columns with the means replacing the `NaN`'s to our dataframe.

In [ ]:
# copy the weather dataframe 
mean_df = df.copy() 

# apply mean to the TEMPERATURE and RAIN column
transform_output = imputer.transform(transform_input)
mean_df[['TEMPERATURE','RAIN']] = transform_output
mean_df

Note there are other options for the `strategy` parameter including 'mean', 'median', 'most_frequent', and 'constant'.

## Fill in Missing Values with Nearest Neighbor

Another option for imputing a value to replace missing values is to leverage k-nearest neighbor (KNN), which works quite well in many cases. Essentially, the idea is to find datapoints that are close to the one with the missing value, all fields considered. Those neighboring records are then used to infer an estimate for the missing value. 

Let's bring in the `KNNInputer` and use the 5 nearest neighbors. We will make the weights uniform and we will tell it to not ignore `NaN` values by setting `metric` to `nan_euclidean`. 

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

Since we are inferring based on some other fields, we need to kick out `ID`,`LOCATION_ID`, and `REPORT_CODE` as those are not useful for the KNN model. They are arbitrary values or randomly generated and have no predictive value. We also need to convert the `REPORT_DATE` to a numeric value. Thankfully we are only working with one month, so let's just grab the day of month and that will be our numeric conversion.

In [ ]:
# copy the datafarme and drop two columns that are not useful for modeling
knn_input = df.drop(['ID','REPORT_CODE','LOCATION_ID'],axis=1)

# extract day of month and make that the `REPORT_DATE`
knn_input['REPORT_DATE'] = knn_input['REPORT_DATE'].dt.strftime('%d').astype(int)

# fit the knn model 
imputer.fit(knn_input)
knn_input

Finally, let's copy the dataframe and apply the transformation to the two columns. The `RAIN` values with row indices of 5, 18, and 19 were missing but now are `0.604`, `0.996`, and `0.986` respectively. The missing `TEMPERATURE` value of row index 12 is inferred to be `55.76`. 

In [ ]:
# copy the dataframe
knn_output = df.copy()

# apply knn transform to the input
knn_transform = imputer.transform(knn_input)

# apply only the TEMPERATURE and RAIN columns back to the dataframe
knn_output.loc[:,["TEMPERATURE","RAIN"]] = knn_transform[:,[1,3]]
knn_output

## Exercise

In the code below is a sample of thermostat data. Complete the code below so the median is imputed for the `temperature` and `humidity` fields' missing values. 

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.DataFrame({
    "record_id" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperature" : [65.2, 65.2, None, 57.2, 57.4, None, 27.5], 
    "humidity" : [.8, None, .7, .6, .7, .7, .8]
})

# create and fit imputer 
imputer = ?
transform_input = df[['temperature','humidity']]
imputer.fit(?)

# apply mean to the temperature and humidity column
transform_output = imputer.transform(?)
df[['temperature','humidity']] = transform_output
df

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.DataFrame({
    "record_id" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperature" : [65.2, 65.2, None, 57.2, 57.4, None, 27.5], 
    "humidity" : [.8, None, .7, .6, .7, .7, .8]
})

# create and fit imputer 
imputer = SimpleImputer(strategy='median')
transform_input = df[['temperature','humidity']]
imputer.fit(transform_input)

# apply mean to the TEMPERATURE and RAIN column
transform_output = imputer.transform(transform_input)
df[['temperature','humidity']] = transform_output
df